In [1]:
### Run sementic segmentation model U-net with pretrained weights ###
import os
os.chdir('E:/BasW/Notebooks') 
work_directory = os.getcwd()
os.chdir(work_directory+'/Scripts')
from pathlib import Path
from DataPreprocessing import *
from DataCreation import N2000_Data
from Unet import unet
import CreateResults as cr
import tensorflow
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as keras
os.chdir('../..') # --> ensures that work directory = E:/BasW/Notebooks
work_directory = os.getcwd()

Using TensorFlow backend.


Set variables and directories

In [2]:
# Set variables and path directories
image_size = (512, 512) 
cell_size = 0.25 
epsg = 28992    
path_training_data = work_directory + "/Data/testing"
path_mask_data_binair = work_directory 
path_original_data = work_directory
h5Path = work_directory + "/Data/h5130/testing/H5130_Drenthe.h5"
json_path = work_directory + "/Data/h5130/testing/H5130_Drenthe.json"
stats_file = work_directory + "/Data/h5130/NormalizationStatsH5130.csv"
geotiff_folder = work_directory + "/Data/h5130/predictions"
model_path = work_directory + '/Data/h5130/checkpoints/JsonModels/Run2_H5130.json'
model_weights = work_directory + '/Data/h5130/checkpoints/Run2_weights_best.h5'
# Create N2000_DataPreparation object
dp = N2000_DataPreparation(image_size, path_training_data, path_mask_data_binair, path_original_data)

Load and normalize test data with training data statistics

In [8]:
# Normalize test dataset
x_test, y_test = dp.ApplyNormalization(h5Path, stats_file)

88.737 25.86 -3.431 6.429


Run the U-net convolutional neural network on unseen data

In [6]:
# Open saved U-net model from json
json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
# Convert Json model to Tensorflow ConvNetmodel
loaded_model = model_from_json(loaded_model_json)
# Load weights into the model
loaded_model.load_weights(model_weights)
print("Loaded model from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk


Predict images and plot predicted images (code on Azure)

In [12]:
# evaluate the model
loaded_model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
scores = loaded_model.evaluate(x_test, y_test, verbose=0, batch_size = 4) 
print("%s: %.2f%%" % (loaded_model.metrics_names[1], scores[1]*100))

acc: 77.15%


In [142]:
# Make predictions and write prediction to geotiff
images, masks, filenames = cr.readH5File(h5Path)
x_test, y_test = dp.ApplyNormalization(h5Path, stats_file)
for i in range(len(x_test)):
    j = i+1
    image = x_test[i:j]
    filename = filenames[i:j][0]
    prediction = loaded_model.predict(image, batch_size = 4)
    prediction = np.squeeze(prediction, axis = 0)
    cr.writeGeoTiff(json_path, prediction, filename, geotiff_folder)


88.737 25.86 -3.431 6.429
23_2018_H5130
61_2018_H5130
14_2018_H5130
24_2017_H5130
4_2018_H5130
74_2017_H5130
20_2017_H5130
70_2017_H5130
14_2016_H5130
61_2017_H5130
0_2018_H5130
5_2018_H5130
48_2017_H5130
70_2016_H5130
39_2016_H5130
62_2018_H5130
40_2018_H5130
53_2016_H5130
39_2018_H5130
62_2016_H5130
18_2018_H5130
19_2018_H5130
36_2017_H5130
40_2016_H5130
39_2017_H5130
32_2018_H5130
10_2016_H5130
34_2018_H5130
31_2017_H5130
30_2016_H5130
67_2016_H5130
72_2017_H5130
17_2016_H5130
48_2018_H5130
0_2017_H5130
33_2016_H5130
24_2018_H5130
21_2017_H5130
47_2018_H5130
12_2018_H5130
36_2018_H5130
37_2018_H5130
57_2018_H5130
20_2016_H5130
72_2018_H5130
50_2018_H5130
51_2018_H5130
18_2016_H5130
10_2017_H5130
46_2018_H5130
0_2016_H5130
11_2018_H5130
28_2018_H5130
63_2018_H5130
74_2018_H5130
11_2016_H5130
53_2018_H5130
50_2016_H5130
46_2016_H5130
14_2017_H5130
37_2017_H5130
21_2018_H5130
63_2017_H5130
4_2016_H5130
17_2018_H5130
12_2016_H5130
29_2017_H5130
49_2017_H5130
12_2017_H5130
70_2018_H5130


In [4]:
cr.vectorizePrediction(geotiff_folder, cat = 'h5130_prob90', prob_treshold = 0.9)

In [ ]:
# SHOW RESULTS ON TEST SET --> OUD --> NOT GEOTIFF

import scipy
predictions = loaded_model.predict(x_test, batch_size = 4)
predictions = predictions[0:20]

for i in range(len(predictions)):
    pred =  predictions[i].reshape(512,512)
    img = x_test[i]
    gt = y_test[i].reshape(512,512)

    scipy.misc.imsave(geotiff_folder + '/Test_img_' +str(i)+".tif", img)
    scipy.misc.imsave(geotiff_folder + '/Test_gt_' + str(i)+".tif", gt)
    scipy.misc.imsave(geotiff_folder + '/Test_prediction_'+str(i)+".tif", pred)